In [2]:
import numpy as np
from sklearn.cross_validation import train_test_split

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.callbacks import LearningRateScheduler

from matplotlib import pyplot as plt
%matplotlib inline

import cv2


Using Theano backend.
Using gpu device 0: GeForce GTX TITAN X (CNMeM is enabled with initial size: 10.0% of memory, cuDNN 5005)


In [3]:
nb_classes = 43
img_size = 32

## loading training data
import pickle
training_set = pickle.load(open("/home/aiml_test_user/training_set.pkl", "rb"))

## transorforming tha data
# resizing all images to one
resized_img=[]
class_labels = []
for i in range(0, len(training_set)):
    sample = training_set[i]
    image = cv2.resize(sample['img'],(img_size, img_size))
    resized_img.append(image)
    class_labels.append(sample['class'])
    
resize_imgs = np.array(resized_img)

Y = np.array(class_labels)

## dividing by 255
X = np.array(resized_img, dtype='float32')
X /= 255

## swapping axes to input into model
X = X.swapaxes(1,3)
X.shape


## splitting the data into train and test
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)


In [6]:
## model

def cnn_model():
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, border_mode='valid', input_shape=(3, img_size, img_size), activation='relu'))
    model.add(Convolution2D(32, 3, 3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu'))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(nb_classes, activation='softmax'))
    return model

model = cnn_model()


lr = 0.01
sgd = SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
          optimizer=sgd,
          metrics=['accuracy'])


def lr_schedule(epoch):
    return lr*(0.1**int(epoch/10))

In [8]:
## fitting the model
batch_size = 30
nb_epoch = 10

model.fit(x_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(x_test, Y_test))

Train on 27446 samples, validate on 11763 samples
Epoch 1/10
27446/27446 [==============================] - 5s - loss: 2.2749 - acc: 0.3529 - val_loss: 0.4749 - val_acc: 0.8464
Epoch 2/10
27446/27446 [==============================] - 5s - loss: 0.4069 - acc: 0.8706 - val_loss: 0.1085 - val_acc: 0.9691
Epoch 3/10
27446/27446 [==============================] - 5s - loss: 0.1838 - acc: 0.9424 - val_loss: 0.0694 - val_acc: 0.9794
Epoch 4/10
27446/27446 [==============================] - 5s - loss: 0.1239 - acc: 0.9623 - val_loss: 0.0394 - val_acc: 0.9894
Epoch 5/10
27446/27446 [==============================] - 5s - loss: 0.0935 - acc: 0.9714 - val_loss: 0.0394 - val_acc: 0.9889
Epoch 6/10
27446/27446 [==============================] - 5s - loss: 0.0800 - acc: 0.9761 - val_loss: 0.0318 - val_acc: 0.9915
Epoch 7/10
27446/27446 [==============================] - 5s - loss: 0.0597 - acc: 0.9825 - val_loss: 0.0328 - val_acc: 0.9912
Epoch 8/10
27446/27446 [==============================] - 5s 

In [9]:
validation = model.evaluate(x_test, Y_test, verbose=1)
print('Test accuracy:', validation[1])

11744/11763 [============================>.] - ETA: 0sTest accuracy: 0.993454050837


In [10]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_7 (Convolution2D)  (None, 32, 30, 30)    896         convolution2d_input_2[0][0]      
____________________________________________________________________________________________________
convolution2d_8 (Convolution2D)  (None, 32, 28, 28)    9248        convolution2d_7[0][0]            
____________________________________________________________________________________________________
maxpooling2d_4 (MaxPooling2D)    (None, 32, 14, 14)    0           convolution2d_8[0][0]            
____________________________________________________________________________________________________
dropout_5 (Dropout)              (None, 32, 14, 14)    0           maxpooling2d_4[0][0]             
___________________________________________________________________________________________

In [4]:
## testing
import pickle
testing_set = pickle.load(open("/home/aiml_test_user/testing_set.pkl", "rb"))

## preprocessing
resized_img_test=[]
for i in range(0, len(testing_set)):
    sample = testing_set[i]
    image = cv2.resize(sample['img'],(img_size, img_size))
    resized_img_test.append(image)
resize_imgs_test = np.array(resized_img_test)

X_test = np.array(resize_imgs_test, dtype='float32')
X_test /= 255

X_test= X_test.swapaxes(1,3)
X_test.shape

(12630, 3, 32, 32)

In [12]:
## predicting the test data
y_pred = model.predict_classes(X_test)

12480/12630 [============================>.] - ETA: 0s

In [15]:
## saving the predicted data
np.savetxt("submission.csv", y_pred, delimiter=",")

In [5]:
x_train.shape

(27446, 3, 32, 32)

In [5]:
## data augmentation
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2)

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(Y_train, nb_classes)
Y_val = np_utils.to_categorical(Y_val, nb_classes)

datagen = ImageDataGenerator(featurewise_center=False, 
                            featurewise_std_normalization=False, 
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            zoom_range=0.2,
                            shear_range=0.1,
                            rotation_range=10.,)

datagen.fit(X_train)

In [7]:
# Reinstallise models 

model = cnn_model()
# let's train the model using SGD + momentum (how original).
lr = 0.01
sgd = SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
          optimizer=sgd,
          metrics=['accuracy'])


def lr_schedule(epoch):
    return lr*(0.1**int(epoch/10))

In [10]:
nb_epoch = 20
batch_size = 40
model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
                            samples_per_epoch=X_train.shape[0],
                            nb_epoch=nb_epoch,
                            validation_data=(X_val, Y_val))

Epoch 1/20
31367/31367 [==============================] - 13s - loss: 2.8337 - acc: 0.2080 - val_loss: 1.6599 - val_acc: 0.4582
Epoch 2/20
31367/31367 [==============================] - 11s - loss: 1.3717 - acc: 0.5470 - val_loss: 0.6300 - val_acc: 0.7744
Epoch 3/20
31367/31367 [==============================] - 10s - loss: 0.6774 - acc: 0.7735 - val_loss: 0.1333 - val_acc: 0.9559
Epoch 4/20
31367/31367 [==============================] - 10s - loss: 0.3907 - acc: 0.8738 - val_loss: 0.1088 - val_acc: 0.9588
Epoch 5/20
31367/31367 [==============================] - 10s - loss: 0.2664 - acc: 0.9155 - val_loss: 0.0392 - val_acc: 0.9893
Epoch 6/20
31367/31367 [==============================] - 11s - loss: 0.2168 - acc: 0.9315 - val_loss: 0.0351 - val_acc: 0.9906
Epoch 7/20
31367/31367 [==============================] - 13s - loss: 0.1701 - acc: 0.9473 - val_loss: 0.0401 - val_acc: 0.9887
Epoch 8/20
31367/31367 [==============================] - 15s - loss: 0.1392 - acc: 0.9563 - val_loss: 0

In [11]:
## predicting the test data
y_pred = model.predict_classes(X_test)

## saving the predicted data
np.savetxt("submission.csv", y_pred, delimiter=",")

12448/12630 [============================>.] - ETA: 0s

In [13]:
## data augmentation - part 2
X_train, X_val, Y_train,Y_val = train_test_split(X, Y, test_size=0.2)

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(Y_train, nb_classes)
Y_val = np_utils.to_categorical(Y_val, nb_classes)

datagen1 = ImageDataGenerator(featurewise_center=True,
                             featurewise_std_normalization=True,
                             rotation_range=20,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             horizontal_flip=True)


# compute quantities required for featurewise normalization
datagen1.fit(X_train)

# Reinstallise models 

model = cnn_model()
# let's train the model using SGD + momentum (how original).
lr = 0.01
sgd = SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
          optimizer=sgd,
          metrics=['accuracy'])


def lr_schedule(epoch):
    return lr*(0.1**int(epoch/10))


nb_epoch = 20
batch_size = 40
model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
                            samples_per_epoch=X_train.shape[0],
                            nb_epoch=nb_epoch,
                            validation_data=(X_val, Y_val))

Epoch 1/20
31367/31367 [==============================] - 10s - loss: 2.8136 - acc: 0.2043 - val_loss: 1.6052 - val_acc: 0.4721
Epoch 2/20
31367/31367 [==============================] - 10s - loss: 1.2541 - acc: 0.5889 - val_loss: 0.3491 - val_acc: 0.8823
Epoch 3/20
31367/31367 [==============================] - 10s - loss: 0.5423 - acc: 0.8230 - val_loss: 0.1078 - val_acc: 0.9668
Epoch 4/20
31367/31367 [==============================] - 11s - loss: 0.3212 - acc: 0.8943 - val_loss: 0.0634 - val_acc: 0.9813
Epoch 5/20
31367/31367 [==============================] - 10s - loss: 0.2366 - acc: 0.9243 - val_loss: 0.0430 - val_acc: 0.9875
Epoch 6/20
31367/31367 [==============================] - 10s - loss: 0.1841 - acc: 0.9420 - val_loss: 0.0302 - val_acc: 0.9904
Epoch 7/20
31367/31367 [==============================] - 10s - loss: 0.1612 - acc: 0.9492 - val_loss: 0.0187 - val_acc: 0.9941
Epoch 8/20
31367/31367 [==============================] - 10s - loss: 0.1367 - acc: 0.9572 - val_loss: 0

In [ ]:
## predicting the test data
y_pred = model.predict_classes(X_test)

## saving the predicted data
np.savetxt("submission.csv", y_pred, delimiter=",")